In [2]:
import fastf1
import pandas as pd
from pathlib import Path
from tqdm import tqdm

# Setup cache and data directories
Path('fastf1_cache').mkdir(exist_ok=True)
fastf1.Cache.enable_cache('fastf1_cache')
DATA_DIR = Path("data")
DATA_DIR.mkdir(exist_ok=True)

YEARS = range(2022, 2026)

races_all, results_all, qualifying_all, laps_all = [], [], [], []
drivers_map, constructors_map, circuits_map = {}, {}, {}

for year in tqdm(YEARS, desc="Seasons"):
    try:
        schedule = fastf1.get_event_schedule(year)
    except Exception as e:
        print(f"Skipping year {year} - could not load schedule: {e}")
        continue

    for _, race in schedule.iterrows():
        round_ = race['RoundNumber']
        race_name = race['EventName']
        circuit = race['Location']
        date = race['EventDate']

        # Skip future races
        if pd.to_datetime(date) > pd.Timestamp.now():
            continue

        # Save race info
        races_all.append({
            "season": year,
            "round": round_,
            "race_name": race_name,
            "circuit": circuit,
            "date": date
        })
        circuits_map[circuit] = {"name": circuit, "location": race['Location'], "country": race['Country']}

        # Load race session
        try:
            race_session = fastf1.get_session(year, round_, 'R')
            race_session.load(laps=True, telemetry=True)
        except Exception as e:
            print(f"Skipping {race_name} ({year}) - race data unavailable: {e}")
            continue

        # Results
        res_df = race_session.results.reset_index()
        for _, r in res_df.iterrows():
            results_all.append({
                "season": year,
                "round": round_,
                "driver": r['Abbreviation'],
                "team": r['TeamName'],
                "position": r['Position'],
                "laps": r['Laps'],
                "time": r['Time'],
                "points": r['Points'],
                "fastest_lap": r.get('FastestLap', None)
            })
            drivers_map[r['Abbreviation']] = {"driver": r['FullName'], "team": r['TeamName']}
            constructors_map[r['TeamName']] = {"team": r['TeamName']}

        # Qualifying
        try:
            qual_session = fastf1.get_session(year, round_, 'Q')
            qual_session.load(results=True, laps=False, telemetry=False)
            qual_df = qual_session.results.reset_index()
            for _, q in qual_df.iterrows():
                qualifying_all.append({
                    "season": year,
                    "round": round_,
                    "driver": q['Abbreviation'],
                    "team": q['TeamName'] if 'TeamName' in q else q.get('Team', None),
                    "position": q['Position'],
                    "q1": q.get('Q1', None),
                    "q2": q.get('Q2', None),
                    "q3": q.get('Q3', None)
                })
        except Exception:
            # If quali not available, fallback: use race grid positions
            for _, r in res_df.iterrows():
                # some result tables use 'Grid' and others 'GridPosition' — try both
                pos = r.get('Grid', r.get('GridPosition', None))
                if pd.isna(pos):
                    pos = None
                qualifying_all.append({
                    "season": year,
                    "round": round_,
                    "driver": r['Abbreviation'],
                    "team": r['TeamName'],
                    "position": pos,  # fallback to grid start
                    "q1": None,
                    "q2": None,
                    "q3": None
                })
            print(f"Qualifying data missing for {race_name} ({year}) - using grid fallback")

        # Laps
        laps = race_session.laps.copy()
        laps['season'] = year
        laps['round'] = round_
        laps_all.append(laps)

# Save CSVs
pd.DataFrame(races_all).to_csv(DATA_DIR/"races_2022_2025.csv", index=False)
pd.DataFrame(results_all).to_csv(DATA_DIR/"results_2022_2025.csv", index=False)
pd.DataFrame(qualifying_all).to_csv(DATA_DIR/"qualifying_2022_2025.csv", index=False)
if laps_all:
    pd.concat(laps_all).to_csv(DATA_DIR/"laps_2022_2025.csv", index=False)
pd.DataFrame(list(drivers_map.values())).to_csv(DATA_DIR/"drivers_2022_2025.csv", index=False)
pd.DataFrame(list(constructors_map.values())).to_csv(DATA_DIR/"constructors_2022_2025.csv", index=False)
pd.DataFrame(list(circuits_map.values())).to_csv(DATA_DIR/"circuits_2022_2025.csv", index=False)

print("✅ All CSVs saved successfully.")


Seasons:   0%|          | 0/4 [00:00<?, ?it/s]core           INFO 	Loading data for Bahrain Grand Prix - Race [v3.6.1]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...


Skipping Pre-Season Track Session (2022) - race data unavailable: Cannot get testing event by round number!
Skipping Pre-Season Test (2022) - race data unavailable: Cannot get testing event by round number!


req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core        WARNING 	Driver 16 completed the race distance 00:00.050000 before the recorded end of the session.
core           INFO 	Finished loading data for 20 drivers: ['16', '55', '44', '63', '20', '77', '31', '22', '14', '24', '47', '18', '23', '3', '4', '6', '27', '11', '1', '10']
core           INFO 	Loading data for Saudi Arabian Grand Prix - Race [v3.6.1]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_

Qualifying data missing for Bahrain Grand Prix (2022) - using grid fallback


core        WARNING 	Failed to perform lap accuracy check - all laps marked as inaccurate (driver 22)
core        WARNING 	Failed to perform lap accuracy check - all laps marked as inaccurate (driver 47)
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '16', '55', '11', '63', '31', '4', '10', '20', '44', '24', '27', '18', '23', '77', '14', '3', '6', '22', '47']
core           INFO 	Loading data for Australian Grand Prix - Race [v3.6.1]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using

...

req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for driver_info. Loading data...
_api           INFO 	Fetching driver list...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for session_status_data. Loading data...
_api           INFO 	Fetching session status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for lap_count. Loading data...
_api           INFO 	Fetching lap count data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for track_status_data. Loading data...
_api           INFO 	Fetching track status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for _extended_timing_data. Loading data...
_api           INFO 	Fetching timing data...
_api           INFO 	Parsing timing data...
req            INFO 	Data has been written to cache!

Qualifying data missing for Azerbaijan Grand Prix (2025) - using grid fallback


req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for session_status_data. Loading data...
_api           INFO 	Fetching session status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for lap_count. Loading data...
_api           INFO 	Fetching lap count data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for track_status_data. Loading data...
_api           INFO 	Fetching track status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for _extended_timing_data. Loading data...
_api           INFO 	Fetching timing data...
_api           INFO 	Parsing timing data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for timing_app_data. Loading data...
_api           INFO 	Fetching timing app data...
req            INFO 	Data has been written t

Qualifying data missing for Singapore Grand Prix (2025) - using grid fallback


✅ All CSVs saved successfully.
